# Data Transformation and Preparation

In [67]:
import pandas as pd
import ast

# read the data
spotify = pd.read_csv('data/Hot 100 Audio Features.csv')
billboard = pd.read_csv('data/Hot Stuff.csv')

# join on song id
spotify = spotify.drop_duplicates(subset=['SongID']) # drop duplicates in spotify
df = pd.merge(spotify, billboard, on=['SongID'], how='right', validate='one_to_many')
df.drop(columns=['index_y', 'index_x', 'Performer_y', 'Song_y'], inplace = True) # get rid of index and duplicate columns

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327895 entries, 0 to 327894
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   SongID                     327895 non-null  object 
 1   Performer_x                327817 non-null  object 
 2   Song_x                     327817 non-null  object 
 3   spotify_genre              313371 non-null  object 
 4   spotify_track_id           284728 non-null  object 
 5   spotify_track_preview_url  168700 non-null  object 
 6   spotify_track_duration_ms  284728 non-null  float64
 7   spotify_track_explicit     284728 non-null  object 
 8   spotify_track_album        284666 non-null  object 
 9   danceability               284170 non-null  float64
 10  energy                     284170 non-null  float64
 11  key                        284170 non-null  float64
 12  loudness                   284170 non-null  float64
 13  mode                       28

In [68]:
# get numeric columns
num_cols = df.select_dtypes(include='number').columns.to_list()

# transform the data so that each row corresponds to a genre-year combination
# aggregate the numeric columns over that combination
df['Year'] = df['WeekID'].apply(lambda x: x.split('/')[2]) # create year column from weekid
df = df.dropna(subset=['spotify_genre']) # drop rows with no genre
df['spotify_genre'] = df['spotify_genre'].apply(ast.literal_eval) # make genre field a list not string
df = df.explode('spotify_genre') # genre is a list of genres for each song, want one genre for each song
genre_df = df.groupby(['spotify_genre', 'Year']).agg({col: 'mean' for col in num_cols}).reset_index()
song_counts = df.groupby(['spotify_genre', 'Year']).size().reset_index(name='song_count') # get song counts
genre_df = pd.merge(genre_df, song_counts, on=['spotify_genre', 'Year'], how='left') # merge song counts with genre_df
print(genre_df.head())
print(genre_df.info())

            spotify_genre  Year  spotify_track_duration_ms  danceability  \
0  21st century classical  1969                        NaN           NaN   
1              a cappella  1960                        NaN           NaN   
2              a cappella  1962              176153.777778      0.447222   
3              a cappella  1963              174740.423729      0.516186   
4              a cappella  1964              181078.444444      0.607444   

     energy       key   loudness      mode  speechiness  acousticness  ...  \
0       NaN       NaN        NaN       NaN          NaN           NaN  ...   
1       NaN       NaN        NaN       NaN          NaN           NaN  ...   
2  0.175333  4.333333 -18.251556  0.111111     0.049922      0.848778  ...   
3  0.218542  6.067797 -18.188373  1.000000     0.038598      0.717441  ...   
4  0.252333  3.333333 -11.744889  1.000000     0.034522      0.776333  ...   

    valence       tempo  time_signature  spotify_track_popularity  \
0    

# Exploratory Data Analysis